<a href="https://colab.research.google.com/github/caumannerman/ncm_project/blob/master/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EA%B3%BC%EC%A0%9C_%EB%A7%88%EB%B2%95%EA%B3%B5%EC%8B%9D_%EC%98%A4%ED%9B%846%EC%8B%9C33%EB%B6%84(Y).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pykrx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from pykrx import stock
import pandas as pd
import requests
import time
from datetime import datetime

In [4]:
# EV/EBTIDA 가져오기
# date='211230'
# year='2021/12'
def get_EVEBITDA(code, year):
  # print ("in get_EVEBITDA")
  url = 'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A' + code + '&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'

  fr_page = requests.get(url)
  fr_tables = pd.read_html(fr_page.text)

  temp_df = fr_tables[3]
  temp_df = temp_df.set_index(temp_df.columns[0])
  temp_df = temp_df.loc[['EV/EBITDA계산에 참여한 계정 펼치기']]
  temp_df.index = ['EV/EBITDA']
  temp_df = pd.DataFrame({code:temp_df[year]})
  temp_df = temp_df.transpose()
  # print (temp_df)
  return temp_df

In [5]:
# ROIC 가져오기
# date='211230'
# year='2021/12'
def get_ROIC(code, year):
  # print ("in get_ROIC")
  url = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A'+ code +'&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

  fr_page = requests.get(url)
  fr_tables = pd.read_html(fr_page.text)

  temp_df = fr_tables[0]
  temp_df = temp_df.set_index(temp_df.columns[0])
  temp_df = temp_df.loc[['ROIC계산에 참여한 계정 펼치기']]
  temp_df.index = ['ROIC']
  temp_df = pd.DataFrame({code:temp_df[year]})
  temp_df = temp_df.transpose()
  print (temp_df)
  return temp_df

# print (get_ROIC('095570',year))

In [6]:
# 종목코드, 종목명 가져오기
# 2022년 1년간의 종목 수익률을 파악해야 하므로 종목 추출 시점은 2021년 연말이 되어야 함
# 따라서 종목, EV/EBTIDA, ROIC 모두 2021년 4분기 기준값으로 가져오기
date='211230'
year='2021/12'
codes = stock.get_market_ticker_list(date, market='ALL')
corp = []

for code in codes:
  name = stock.get_market_ticker_name(code)
  corp.append([code, name])

df = pd.DataFrame(data=corp, columns=['종목코드', '종목명'])
df_code = df['종목코드']

# EV/EBITDA dataframe 생성 및 적재
# 첫번재 종목이 3월 결산 종목이라 2021년 12월 데이터 값이 홈페이지에 나와있지 않음 -> 삼성전자를 임의로 첫번째 값으로 입력
evebitda_df=get_EVEBITDA('005930',year)

for num, code in enumerate(df_code):  
  if num==5:
      break
  try:
      if num==1:
        return_df=get_EVEBITDA(code,year)
        evebitda_df=pd.concat([evebitda_df, return_df])
        print (evebitda_df)
        evebitda_df=evebitda_df.drop(index='005930')  
      else:
        return_df=get_EVEBITDA(code,year)
        evebitda_df=pd.concat([evebitda_df, return_df])
  except:
      pass
print (evebitda_df)

# ROIC dataframe 생성 및 적재
roic_df=get_ROIC('005930',year)

for num, code in enumerate(df_code):  
  if num==5:
      break
  try:
      if num==1:
        return_df=get_ROIC(code,year)
        roic_df=pd.concat([roic_df, return_df])
        print (roic_df)
        roic_df=roic_df.drop(index='005930')  
      else:
        return_df=get_ROIC(code,year)
        roic_df=pd.concat([roic_df, return_df])
  except:
      pass
print (roic_df)


       EV/EBITDA
005930      6.23
095570      4.85
       EV/EBITDA
095570      4.85
006840     34.61
054620    274.52
265520      5.27
        ROIC
005930  23.9
       ROIC
095570  2.0
        ROIC
005930  23.9
095570   2.0
        ROIC
006840  -5.2
        ROIC
054620  -8.1
        ROIC
265520  37.9
        ROIC
095570   2.0
006840  -5.2
054620  -8.1
265520  37.9


In [84]:
magic = pd.concat([evebitda_df,roic_df],axis=1)
print(magic)


       EV/EBITDA  ROIC
095570      4.85   2.0
006840     34.61  -5.2
054620    274.52  -8.1
265520      5.27  37.9


In [101]:
# 여기부터 추가된 코드
# ROIC와 EV/EBITDA 열이 모두 String타입이었는데 이를 float타입으로 바꾼다 ( 대소관계를 따져 정렬해야하므로)
magic22 = magic.astype({'ROIC':'float', 'EV/EBITDA':'float'})
# EV/EBITDA 기준으로 오름차순 정렬
magic22 = magic22.sort_values('EV/EBITDA', ascending=True)
print(magic22)
# EV/EBITDA 기준으로 오름차순 정렬되어있으므로, 해당순서대로 EV/EBITDA rank값을 할당해줌 
magic22['EV/EBITDA_rank'] = [i for i in range(1, len(magic22) + 1)]
print(magic22)

roic_arr = []

# 인덱스( 종목 번호)와 roic값을 tuple형태로 넣어줌 
for idx, row in magic22.iterrows():
    # print("** index name:", idx)
    # print(idx, row["ROIC"])
    roic_arr.append((row["ROIC"], idx))

# roic 값을 기준으로 하여 내림차순 정렬 
roic_arr = sorted(roic_arr, reverse = True)
# 인덱스 (종목번호)가 원래의 순서 그대로 담긴 list를 생성 
iiidx_arr = list(magic22.index)
print(roic_arr)

# 여기부터 ROIC_rank 에 차례대로 값을 담을 것
ROIC_rank_arr = [0] * len(iiidx_arr)
for idx, (now_roic, secu_num) in enumerate(roic_arr):
  # print(iiidx_arr.index(secu_num))
  # print(idx)
  ROIC_rank_arr[iiidx_arr.index(secu_num)] = idx + 1

# 결과적으로 ROIC_rank_arr에 roic 랭크가 담김
print(ROIC_rank_arr)

magic22['ROIC_rank'] = ROIC_rank_arr

print(magic22)

# EBITDA_rank와 ROIC_rank를 합쳐서 result_rank를 새로운 열에 저장 
magic22['result_rank'] = magic22['EV/EBITDA_rank'] + magic22['ROIC_rank']
print(magic22)
# 끗!!!!!!!!!!
# 끗!!!!!!!!!!
# 끗!!!!!!!!!!
# 끗!!!!!!!!!!

        EV/EBITDA  ROIC
095570       4.85   2.0
265520       5.27  37.9
006840      34.61  -5.2
054620     274.52  -8.1
        EV/EBITDA  ROIC  EV/EBITDA_rank
095570       4.85   2.0               1
265520       5.27  37.9               2
006840      34.61  -5.2               3
054620     274.52  -8.1               4
[(37.9, '265520'), (2.0, '095570'), (-5.2, '006840'), (-8.1, '054620')]
[2, 1, 3, 4]
        EV/EBITDA  ROIC  EV/EBITDA_rank  ROIC_rank
095570       4.85   2.0               1          2
265520       5.27  37.9               2          1
006840      34.61  -5.2               3          3
054620     274.52  -8.1               4          4
        EV/EBITDA  ROIC  EV/EBITDA_rank  ROIC_rank  result_rank
095570       4.85   2.0               1          2            3
265520       5.27  37.9               2          1            3
006840      34.61  -5.2               3          3            6
054620     274.52  -8.1               4          4            8


In [76]:
# def sortDataFrame(df):
#   code_EE_list = []
#   code_RO_list = []
#    # 일단 df에 EV/EBITDA_rank, ROIC_rank, Final_rank 열을 추가해야한다.
#   df['EV/EBITDA_rank'] = [i for i in range(1, len(df) + 1)]
#   df['ROIC_rank'] = code_RO_list
#   df['Final_rank'] = 0

  
#   # 매개변수로 전달받은 DataFrame의 각 row를 돌며  
#   # ee_list에는 (종목코드(문자열), EV/EBITDA (float)) 와 같이 튜플 형태로 쌓아줌
#   # for idx, row in df.iterrows():
#   #   df['EV/EBITDA_rank']
#   #   code_EE_list.append((row["EV/EBITDA"], idx))
#   #   code_RO_list.append((row["ROIC"], idx))

 
#   # df.loc['095570']['EV/EBITDA_rank'] = (1.1, "lll")
#   return df
#   # return code_EE_list, code_RO_list



In [8]:
magic['evebitda_rank'] = magic['evebitda'].rank()
magic['roic_rank'] = magic['roic'].rank(ascending=False)
magic['total'] = magic['evebitda'] + magic['roic_rank']
magic['total_rank'] = magic['total'].rank()
magic = magic.sort_values(by=['total_rank'])
 
today = datetime.today().strftime('%y-%m-%d')
magic.to_excel('Magic(' + 'today' + ').xlsx')
 
return magic

print(make_magic_df(year, date))

KeyError: ignored